Download Dataset

In [ ]:
! pip install --user kaggle
! sudo apt install unzip

In [ ]:
! mkdir /root/.kaggle
! touch /root/.kaggle/kaggle.json
! echo '{"username":"alexdean","key":"ee3a7d6203ab285a89ce6ab4f9d88695"}' > /root/.kaggle/kaggle.json
! kaggle datasets download lexset/synthetic-asl-alphabet

In [ ]:
! unzip synthetic-asl-alphabet.zip

Import Necessary Libraries

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import os
from tqdm import tqdm
import string

Prepare Images

In [5]:
def rotate(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  rot_img = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return rot_img

In [23]:
img_size = 100
data_dir = "Train_Alphabet/"
X = []
y = []
i = -1
alphabet = list(string.ascii_uppercase)
alphabet.append('Blank')
for folder in alphabet:
    print(folder)
    i+=1
    img_num = -45
    for img in tqdm(os.listdir(data_dir+folder)):
        img_num += 1
        if img_num >= 46:
          img_num = -45
        img_file = cv2.imread(data_dir+folder+'/'+img)
        img_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_file, (img_size, img_size))
        img_resized = rotate(img_resized, img_num*2)
        X.append(img_resized)
        y.append(i)
X = np.asarray(X)
y = np.asarray(y)

A


100%|██████████| 900/900 [00:09<00:00, 94.65it/s] 


B


100%|██████████| 900/900 [00:09<00:00, 94.12it/s] 


C


...

100%|██████████| 900/900 [00:09<00:00, 98.57it/s] 


Z


100%|██████████| 900/900 [00:09<00:00, 98.63it/s] 


Blank


100%|██████████| 900/900 [00:08<00:00, 104.61it/s]


In [24]:
x_norm=tf.keras.utils.normalize(X, axis=1)

CNN

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.25, shuffle= True)


#Instantiation
model = Sequential()

#1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(img_size,img_size,1), kernel_size=(11,11), strides=(4,4), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))

#4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))

#5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(32,32,3,)))
model.add(BatchNormalization())
model.add(Activation('elu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

#2nd Fully Connected Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('elu'))
#Add Dropout
model.add(Dropout(0.4))

#3rd Fully Connected Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('elu'))
#Add Dropout
model.add(Dropout(0.4))

#Output Layer
model.add(Dense(27))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 25, 25, 96)        11712     
                                                                 
 batch_normalization_9 (Batc  (None, 25, 25, 96)       384       
 hNormalization)                                                 
                                                                 
 activation_9 (Activation)   (None, 25, 25, 96)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 13, 13, 256)       614656    
                                                                 
 batch_normalization_10 (Bat  (None, 13, 13, 256)     

Train the Neural Net

In [27]:
model.fit(x_train, y_train, batch_size = 20, epochs = 30, validation_freq=1, validation_data=(x_test, y_test))

Epoch 1/30
912/912 [==============================] - 28s 28ms/step - loss: 2.9629 - accuracy: 0.1356 - val_loss: 2.6631 - val_accuracy: 0.1939
Epoch 2/30
912/912 [==============================] - 25s 28ms/step - loss: 2.3839 - accuracy: 0.2733 - val_loss: 2.5177 - val_accuracy: 0.2440
Epoch 3/30
912/912 [==============================] - 25s 27ms/step - loss: 2.0413 - accuracy: 0.3650 - val_loss: 2.7495 - val_accuracy: 0.2390
Epoch 4/30
912/912 [==============================] - 25s 27ms/step - loss: 1.7517 - accuracy: 0.4464 - val_loss: 1.8878 - val_accuracy: 0.4048
Epoch 5/30
912/912 [==============================] - 25s 27ms/step - loss: 1.4953 - accuracy: 0.5272 - val_loss: 1.5732 - val_accuracy: 0.5119
Epoch 6/30
912/912 [==============================] - 25s 28ms/step - loss: 1.2610 - accuracy: 0.6024 - val_loss: 1.4771 - val_accuracy: 0.5524
Epoch 7/30
912/912 [==============================] - 25s 27ms/step - loss: 1.0378 - accuracy: 0.6793 - val_loss: 1.3116 - val_accuracy:

In [34]:
model.save('model.h5')